In [1]:
import pandas as pd

df_geo = pd.read_csv('./data/geolocalizacion.csv', sep=';')
df_car = pd.read_csv('./data/caracteristicas.csv', sep=';')
df_web = pd.read_csv('./data/web.csv', sep=';')

In [2]:
df_meta_geo = df_geo[['rk', 'latitud', 'longitud', 'codigo_postal', 'pais', 'provincia', 'ciudad', 'distrito', 'barrio', 'calle', 'planta']]
"""
Desechamos las columnas de metadatos que no van a influir en nuestro knn. 
Esta información, en el caso de llegar a representarla lo que haremos será enriquecer con una api en una tabla exclusiva para ello 
ennuestra BBD


necesario:  
            [rk] necesitamos el identificador del piso.
            [planta] es un valor muy necesario para nuestro  algoritmo

útilies:   
            [geo] no es necesario si los barrios estuviesen bien definidios. pero como aún no tenemos clara la manera de adquirir el barrio lo mantenemos.
            [CP] si viéramos que los barrios son una lista de codigos postales podríamos ahorrarnos el uso del shapefile y simplemente relacionar CP con barrios. 
            [Pais] provincia y ciudad podrían ser útiles si están bien metidos los datos.
            [distrito y barrio] están un poco sucios pero los mantenemmos aunque luego lo machaquemos con nuestraa información.df_geo
            [calle] podría no ser necesario para nuestro algoritmo pero por ahora lo mantenemos, seguramente lo desechemos más adelante.
"""
df_meta_geo.head(3)


,rk,latitud,longitud,codigo_postal,pais,provincia,ciudad,distrito,barrio,calle,planta
0,UV/2020/000001,403.858,-372.599,28019,1,1,Madrid,Carabanchel,NaN,Avenida de Oporto,1
1,UV/2020/000003,404.278,-363.083,28037,1,1,Madrid,San Blas,Canillejas,Avenida Institucion Libre de Enseñanza,9
2,UV/2020/000004,401.924,-366.922,28341,1,1,Valdemoro,Zona estación,NaN,Calle Tenerife,1


In [3]:
df_meta_car=df_car[['uv', 'estado', 'habitaciones', 'banos','metros','fecha_construccion','orientacion','situacion', 'certificado_energetico', 'portal', 'ascensor', 'parque_infantil', 'terraza', 'trastero', 'piscina_comunitaria', 'cuarto_de_basura','zonas_comunes', 'piscina', 'garaje', 'tejado', 'calefaccion', 'certificaciones', 'jardin', 'aire_acondicionado', 'seguro_hogar']]
"""
Desechamos las columnas de datos que no crremos que nos vayan a influir.complex

eliminamos: 
            [rk] no necesitamos saber en una misma columna el estado del piso y su identificador. lo tenemos cada uno en su columna correspondiente. Posteriormente filtraremos aquellos pisos repetidos que por haber sido reformados nos siguen apareciendo sin reformar.Además tendremos que ser consistenetes con la nomenclartura de esta columna ya que rk de la tabla de geo y uv de esta tabla contienen la misma información, no es así con rk de esta tabla.complex

            [metrica] es inconsistente que tengamos una columna "metros" y redundemos en que la métrica es m2. Asumimos que todos los inmuebles están en metros y que si fuese necesario incluir en medidas imperiales (por ejemplo) se transformaría a metros o se transformaría a metros y se incluría una columna con la información en yardas en todas las viviendas.complex

mantenemos: 
            Muy importante para nuestro algoritmo
            [ascensor ]

            Podría ser interesante para nuestro algoritmo 
            [fecha_construccion] [terraza] [orientacion] [situacion][portal][parque_infantil] [trastero] [piscina_comunitaria] [piscina] [cuarto_de_basura] [zonas_comunes] [garaje] [tejado] [calefaccion] [certificaciones] [certificado_energetico] [jardin] [aire_acondicionado] [seguro_hogar]
            

Mantenemos muchas columnas poco relevantes pero el poderador les dará su valor.

NOTA: columna  [portal] es si tiene o no portero automático. columana [techo] es si está o no abuardillado 

"""

#puede que algunos estados no sean ni '1' ni '2', consideramos 'otro' como la única opción

df_meta_car.estado[(df_meta_car.estado != '2') & (df_meta_car.estado != '1')] = '?'


#tenemos algunos valores repetidos como:
df_meta_car[df_meta_car.uv == 'UV/2020/000011']

<ipython-input-3-577c4870c7d7>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_meta_car.estado[(df_meta_car.estado != '2') & (df_meta_car.estado != '1')] = '?'
/home/emilio/.local/lib/python3.8/site-packages/pandas/core/series.py:992: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(~key, value, inplace=True)


,uv,estado,habitaciones,banos,metros,fecha_construccion,orientacion,situacion,certificado_energetico,portal,...,cuarto_de_basura,zonas_comunes,piscina,garaje,tejado,calefaccion,certificaciones,jardin,aire_acondicionado,seguro_hogar
9,UV/2020/000011,1,3,1,83,1966.0,Suroeste,Exterior,F,0,...,0,0,0,0,0.0,0,0.0,0,0,0
10,UV/2020/000011,2,3,1,83,1966.0,Suroeste,NaN,F,1,...,0,0,0,0,0.0,0,0.0,0,0,0


In [4]:
df_meta_web=df_web[['uv', 'estado', 'tipo_vivienda', 'precio', 'moneda', 'gastos_comunitarios']]
"""
Mantenemos prácticamente todos los datos.

Alunos comentarios:
    [estado] debería ser redundante ya que lo tenemos tb en df_meta_car pero nos servirá para asegurarnos de que el dato es correcto
    [precio_antiguo] [descuento] los desechamos, nuestro algoritmo sólo considerará el precio actual. si es con descuento o no será el comercial quien tenga esta información y recalcará al clliente que se estaría aprovechando de un descuento.
    [moneda] lo mantenemos sólo por si en un futuro incluyeramos viviendas en otra moneda. si incluyeramos una moneda diferente habría que hacer un cálculo y adaptar todas las viciendas a la misma moneda aunque fuese de manera aproximada. 
    [gastos_comunitarios] esta variable podría ser interesante para nuestro algoritmo así que la mantenemos. en el momento de recomendar una u otra casa puede que sí sea información valiosa para los comerciales: nos convendría vender antes un piso con una cuota de gastos comunitarios alta que una con pocos gastos comunitarios.
"""
df_meta_web.sample(3)

,uv,estado,tipo_vivienda,precio,moneda,gastos_comunitarios
180,UV/2021/000346,1,Piso,364000.0,EURO,153.77
103,UV/2021/000337,1,Piso,280000.0,EURO,219.51
140,UV/2021/000414,1,Piso,110000.0,EURO,0.00


In [5]:
df_meta_geo.columns=['uv', 'latitud', 'longitud', 'codigo_postal', 'pais', 'provincia',
       'ciudad', 'distrito', 'barrio', 'calle', 'planta']
#mantenemmos consistente nuestra nomenclatura del identificador y llamamos a todos los uv correctamente (no rk)
df_meta_geo.sample(3)

,uv,latitud,longitud,codigo_postal,pais,provincia,ciudad,distrito,barrio,calle,planta
24,UV/2020/000026,404.292,-368.492,28006,1,1,Madrid,Salamanca,NaN,Calle Lagasca,6
23,UV/2020/000025,403.933,-367.964,28045,1,1,Madrid,Arganzuela,NaN,Calle Eros,3
252,UV/2020/000307,403.988,-369.541,28045,1,1,Madrid,Arganzuela,NaN,Calle de Cáceres,3


In [6]:
df_meta_car.columns[1:]

Index(['estado', 'habitaciones', 'banos', 'metros', 'fecha_construccion',
       'orientacion', 'situacion', 'certificado_energetico', 'portal',
       'ascensor', 'parque_infantil', 'terraza', 'trastero',
       'piscina_comunitaria', 'cuarto_de_basura', 'zonas_comunes', 'piscina',
       'garaje', 'tejado', 'calefaccion', 'certificaciones', 'jardin',
       'aire_acondicionado', 'seguro_hogar'],
      dtype='object')

In [7]:
"""
 En primer lugar deberemos eliminar las viviendas que, habiendo sido reformadas, nos osiguen apareciendo con la opción sin reformar
"""

#2ddo esta función elimina los duplicados pero se queda con el que tenga el estado más alto y podrían perderse datos. 

df_meta_car = df_meta_car.sort_values('estado').drop_duplicates('uv',keep='last')
"""
    Exixten viviendas repetidas y algunas de ellas tienen valores inconsistentes, por lo tanto vamos a intentar unificar.
"""

df_meta_car.uv.value_counts().max()

1

In [8]:
df_meta_car['NS']=df_meta_car.orientacion
df_meta_car['EO']=df_meta_car.orientacion

In [9]:
def saca_NS(cadena):
    CADENA=cadena.upper()
    if 'NOR' in CADENA: return 'N'
    elif 'SU' in CADENA: return 'S'
    else: return ''

def saca_EO(cadena):
    CADENA=cadena.upper()
    if 'OES' in CADENA: return 'O'
    elif 'EST' in CADENA: return 'E'
    else: return ''

In [10]:
df_meta_car['NS']=df_meta_car['orientacion'].apply(saca_NS)
df_meta_car['EO']=df_meta_car['orientacion'].apply(saca_EO)


In [11]:
df_meta_car.drop(['orientacion'], axis=1, inplace= True)

In [12]:
def string2num(cadena):
    try:
        print(type(cadena)
        return (round(cadena,0))
    except:
        return int(0)


'1.3'

In [13]:

df_meta_car.estado[(df_meta_car.estado != '2') & (df_meta_car.estado != '1')] = '?'
df_meta_car

<ipython-input-13-dce2dbc7d484>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_meta_car.estado[(df_meta_car.estado != '2') & (df_meta_car.estado != '1')] = '?'


,uv,estado,habitaciones,banos,metros,fecha_construccion,situacion,certificado_energetico,portal,ascensor,...,piscina,garaje,tejado,calefaccion,certificaciones,jardin,aire_acondicionado,seguro_hogar,NS,EO
0,UV/2020/000001,1,2,1,53,1963.0,Exterior,F,0,1,...,0,0,0.0,0,0.0,0,0,0,N,
252,UV/2020/000301,1,3,2,80,1983.0,Exterior,F,0,0,...,0,0,0.0,0,0.0,0,0,0,N,O
251,UV/2020/000300,1,4,1,120,1965.0,Interior,F,0,1,...,0,0,0.0,0,0.0,0,0,0,N,O
250,UV/2020/000299,1,2,2,101,1940.0,Exterior,F,1,1,...,0,0,0.0,0,0.0,0,0,0,N,E
249,UV/2020/000297,1,1,1,59,2011.0,Exterior,F,1,1,...,0,1,0.0,0,0.0,0,0,0,N,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264,UV/2020/000307,2,2,2,70,1950.0,Exterior,F,0,1,...,0,0,0.0,0,0.0,0,0,0,,E
266,UV/2020/000309,2,3,1,65,1963.0,Interior,F,0,0,...,0,0,0.0,0,0.0,0,0,0,,O
361,UV/2021/000397,2,2,2,73,1973.0,Exterior,F,1,1,...,0,0,0.0,0,0.0,0,0,0,S,O
293,UV/2020/000330,2,2,1,41,1914.0,Interior,F,1,0,...,0,0,0.0,0,0.0,0,0,0,N,E


In [14]:
df_meta_car[['habitaciones', 'banos', 'fecha_construccion']] = df_meta_car[['habitaciones', 'banos', 'fecha_construccion']].apply(string2num)


0      2
252    3
251    4
250    2
249    1
      ..
264    2
266    3
361    2
293    2
309    2
Name: habitaciones, Length: 371, dtype: int64
0      1
252    2
251    1
250    2
249    1
      ..
264    2
266    1
361    2
293    1
309    2
Name: banos, Length: 371, dtype: int64
0      1963.0
252    1983.0
251    1965.0
250    1940.0
249    2011.0
        ...  
264    1950.0
266    1963.0
361    1973.0
293    1914.0
309    1965.0
Name: fecha_construccion, Length: 371, dtype: float64


In [15]:
df_meta_car

,uv,estado,habitaciones,banos,metros,fecha_construccion,situacion,certificado_energetico,portal,ascensor,...,piscina,garaje,tejado,calefaccion,certificaciones,jardin,aire_acondicionado,seguro_hogar,NS,EO
0,UV/2020/000001,1,2,1,53,1963.0,Exterior,F,0,1,...,0,0,0.0,0,0.0,0,0,0,N,
252,UV/2020/000301,1,3,2,80,1983.0,Exterior,F,0,0,...,0,0,0.0,0,0.0,0,0,0,N,O
251,UV/2020/000300,1,4,1,120,1965.0,Interior,F,0,1,...,0,0,0.0,0,0.0,0,0,0,N,O
250,UV/2020/000299,1,2,2,101,1940.0,Exterior,F,1,1,...,0,0,0.0,0,0.0,0,0,0,N,E
249,UV/2020/000297,1,1,1,59,2011.0,Exterior,F,1,1,...,0,1,0.0,0,0.0,0,0,0,N,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264,UV/2020/000307,2,2,2,70,1950.0,Exterior,F,0,1,...,0,0,0.0,0,0.0,0,0,0,,E
266,UV/2020/000309,2,3,1,65,1963.0,Interior,F,0,0,...,0,0,0.0,0,0.0,0,0,0,,O
361,UV/2021/000397,2,2,2,73,1973.0,Exterior,F,1,1,...,0,0,0.0,0,0.0,0,0,0,S,O
293,UV/2020/000330,2,2,1,41,1914.0,Interior,F,1,0,...,0,0,0.0,0,0.0,0,0,0,N,E
